# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [3]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ['employees', 'salary', 'studies'],
        'definition': ['Employee information, name, ID, department, and hire date',
        'Salary details for each year, including bonuses and deductions',
        'Educational studies, name of the institution, degree, and graduation year']
}
df = pd.DataFrame(data)
print(df)

       table                                         definition
0  employees  Employee information, name, ID, department, an...
1     salary  Salary details for each year, including bonuse...
2    studies  Educational studies, name of the institution, ...


In [4]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [5]:
print(text_tables)

employees: Employee information, name, ID, department, and hire date
salary: Salary details for each year, including bonuses and deductions
studies: Educational studies, name of the institution, degree, and graduation year


In [6]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [7]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(
    tables=text_tables,
    question="Which employees with a computer science degree earned more than $100,000 in 2023?"
    )

In [8]:
print(return_OAI(pqt1))

{
    "tables": ["employees", "salary", "studies"]
}


In [9]:
pqt3 = prompt_question_tables.format(
    tables=text_tables,
    question="What is the average salary of employees hired after 2020 grouped by department?"
    )

In [10]:
print(return_OAI(pqt3))

{
    "tables": ["employees", "salary"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [11]:
# Version 1: Analysis of Table Relationships and Data Linkages
prompt_question_tables2 = """
You are an expert database analyst assisting a user with an employee management database. Below are the tables, their detailed definitions, and their relationships. Use this information to provide a clear, actionable response to the user's question.

###Database Overview
This database tracks employee details, compensation history, and educational qualifications to support HR analytics and career development projects.

###Tables
{tables}

###Relationships Between Tables
- employees to salary: one-to-many (Each employee has multiple salary records over time, linked by employee ID as the primary key in 'employees' and foreign key in 'salary').
- employees to studies: one-to-many (Each employee can have multiple educational qualifications, linked by employee ID as the primary key in 'employees' and foreign key in 'studies').
- salary to studies: no direct relation (These tables connect indirectly through 'employees' via employee ID, but have no direct key linking them).

###User Question
{question}

###Instructions
Analyze the user's question and provide a detailed response in JSON format with the following structure:
- "tables": list of all table names in the database
- "relationships": list of objects, each with:
  - "table1": first table name
  - "table2": second table name
  - "relation_type": type of relationship (e.g., one-to-many, no direct relation)
  - "description": explanation of the relationship and how it’s implemented (e.g., key connections)
- "answer": a concise explanation addressing the user’s question, leveraging the table relationships

Focus on clarity, practical examples, and relevance to the user’s goal. Avoid assumptions beyond the provided schema unless explicitly needed to illustrate a point.
"""

In [ ]:
pqt4 = prompt_question_tables2.format(
    tables=text_tables,
    question="How are the tables linked together, so I can create a career growth tracker showing salary trends and education for each employee?"
)

In [13]:
print(return_OAI(pqt4))

```json
{
    "tables": ["employees", "salary", "studies"],
    "relationships": [
        {
            "table1": "employees",
            "table2": "salary",
            "relation_type": "one-to-many",
            "description": "Each employee in the 'employees' table can have multiple salary records in the 'salary' table. This relationship is established through the employee ID as the primary key in 'employees' and as a foreign key in 'salary'."
        },
        {
            "table1": "employees",
            "table2": "studies",
            "relation_type": "one-to-many",
            "description": "Each employee in the 'employees' table can have multiple educational qualifications in the 'studies' table. This relationship is linked through the employee ID as the primary key in 'employees' and as a foreign key in 'studies'."
        },
        {
            "table1": "salary",
            "table2": "studies",
            "relation_type": "no direct relation",
            "descri

In [14]:
# Version 2: Potential Use Cases and Functional Capabilities of the Database
prompt_question_tables3 = """
You are a seasoned SQL consulting expert assisting a user with their employee management database. Below are the tables, their detailed definitions, and their relationships. Use this information to provide an impressive, actionable response to the user's question about what tasks their database can support.

###Database Overview
This database tracks employee details, compensation history, and educational qualifications to empower HR analytics, workforce planning, and career development initiatives.

###Tables
{tables}

###Relationships Between Tables
- employees to salary: one-to-many (Each employee has multiple salary records over time, linked by employee ID as the primary key in 'employees' and foreign key in 'salary').
- employees to studies: one-to-many (Each employee can have multiple educational qualifications, linked by employee ID as the primary key in 'employees' and foreign key in 'studies').
- salary to studies: no direct relation (These tables connect indirectly through 'employees' via employee ID, but have no direct key linking them).

###User Question
What tasks can my database help me accomplish?

###Instructions
Analyze the database schema and relationships, then provide a detailed response in JSON format with the following structure:
- "tables": list of all table names in the database
- "capabilities": list of objects, each with:
  - "task": a specific task phrased as "Track [X] and identify [Y]" (e.g., "Track salary trends and identify top earners")
  - "description": a practical, impressive explanation of how the database enables this task, using SQL-relevant insights (e.g., joins, aggregations)
Focus on delivering at least three creative, high-value use cases that highlight the database’s potential. Use confident, expert-level language to inspire the user as a top-tier consultant.
"""

In [15]:
pqt5 = prompt_question_tables3.format(
    tables=text_tables,
    question="What tasks can my database help me accomplish?"
)

In [16]:
print(return_OAI(pqt5))

```json
{
    "tables": ["employees", "salary", "studies"],
    "capabilities": [
        {
            "task": "Track employee career progression and identify high-potential individuals",
            "description": "By joining the 'employees' table with the 'salary' table on employee ID, you can track salary changes over time. Using aggregations like MAX(salary_amount) and MIN(salary_amount), you can identify employees with the highest salary growth rates, indicating high-potential individuals deserving of career development opportunities."
        },
        {
            "task": "Analyze workforce education levels and identify skill gaps",
            "description": "By joining the 'employees' table with the 'studies' table on employee ID, you can analyze the educational qualifications of your workforce. Using COUNT(degree) GROUP BY degree, you can identify the most common degrees among employees and pinpoint areas where additional training or recruitment may be needed to fill skill

In [17]:
# Version 3: Types of SQL Queries Supported and Their Applications
prompt_question_tables4 = """
You are an elite SQL consultant guiding a user through their employee management database. Below are the tables, their detailed definitions, and their relationships. Leverage this schema to deliver a standout response to the user's question about SQL query capabilities.

###Database Overview
This database captures employee profiles, salary histories, and educational backgrounds, designed to fuel advanced HR analytics and data-driven decisions.

###Tables
{tables}

###Relationships Between Tables
- employees to salary: one-to-many (Each employee has multiple salary records over time, linked by employee ID as the primary key in 'employees' and foreign key in 'salary').
- employees to studies: one-to-many (Each employee can have multiple educational qualifications, linked by employee ID as the primary key in 'employees' and foreign key in 'studies').
- salary to studies: no direct relation (These tables connect indirectly through 'employees' via employee ID, but have no direct key linking them).

###User Question
What types of SQL queries can my database support?

###Instructions
Examine the database structure and relationships, then provide a detailed response in JSON format with the following structure:
- "tables": list of all table names in the database
- "query_types": list of objects, each with:
  - "query_type": a specific SQL query type (e.g., "Aggregation queries")
  - "description": a compelling explanation of how the database supports this query type, including example use cases and SQL techniques
Offer at least three distinct query types, showcasing the database’s versatility. Use persuasive, expert-level language to demonstrate your SQL mastery and excite the user about their data’s potential.
"""

In [18]:
pqt6 = prompt_question_tables4.format(
    tables=text_tables,
    question="What types of SQL queries can my database support?"
)

In [19]:
print(return_OAI(pqt6))

```json
{
    "tables": ["employees", "salary", "studies"],
    "query_types": [
        {
            "query_type": "Aggregation queries",
            "description": "Your database fully supports aggregation queries, allowing you to analyze and summarize data across tables. For example, you can calculate the average salary of all employees, find the highest bonus given in a year, or count the number of employees with a specific degree. By using SQL functions like AVG(), MAX(), COUNT(), and GROUP BY, you can gain valuable insights into salary trends, educational distributions, and departmental statistics."
        },
        {
            "query_type": "Join queries",
            "description": "With the relational structure in your database, join queries are a powerful tool to combine data from multiple tables. You can retrieve comprehensive employee profiles including salary details and educational backgrounds in a single query. For instance, you can join 'employees' with 'salary' to